In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline
import urllib, json, requests
from shapely.geometry import Point, Polygon
import geopandas as gpd
from shapely import wkt

In [2]:
# f = pd.read_csv('calfires1992-2020.csv', low_memory=False)
# c = gpd.read_file('calfires1992-2020.json')

In [3]:
fires = pd.read_csv('FireOccurance1992-2020.csv', low_memory=False)
counties = gpd.read_file('us-county-boundaries.geojson')


In [4]:
fires

,X,Y,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,-121.005820,40.036939,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,...,1730.0,0.10,A,40.036944,-121.005833,USFS,CA,63,6063.0,Plumas County
1,-120.404431,38.933050,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,...,1530.0,0.25,A,38.933056,-120.404444,USFS,CA,61,6061.0,Placer County
2,-120.735542,38.984161,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,...,2024.0,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17,6017.0,El Dorado County
3,-119.913320,38.559161,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,...,1400.0,0.10,A,38.559167,-119.913333,USFS,CA,3,6003.0,Alpine County
4,-119.933043,38.559161,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,...,1200.0,0.10,A,38.559167,-119.933056,USFS,CA,3,6003.0,Alpine County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303561,-108.959489,39.037884,2303562,400732978,ICS209_2020_11710294,INTERAGCY,IA-ICS209,BLM,USCOGRD,Grand Junction Field Office,...,NaN,1.00,B,39.037890,-108.959500,MISSING/NOT SPECIFIED,CO,NaN,NaN,NaN
2303562,-119.503043,37.148606,2303563,400732979,ICS209_2020_11781527,INTERAGCY,IA-ICS209,ST/C&L,USCAMMU,Merced-Mariposa Unit,...,NaN,100.00,D,37.148611,-119.503056,Private,CA,Madera,6039.0,Madera County
2303563,-114.442787,46.151364,2303564,400732980,ICS209_2020_11815219,INTERAGCY,IA-ICS209,FS,USMTBRF,Bitterroot National Forest,...,NaN,50.00,C,46.151370,-114.442800,MISSING/NOT SPECIFIED,MT,NaN,NaN,NaN
2303564,-120.114486,46.670334,2303565,400732982,ICS209_2020_11831809,INTERAGCY,IA-ICS209,FWS,USWAMCR,Mid Columbia National Wildlife Refuge Complex,...,1900.0,24892.00,G,46.670340,-120.114500,UNDEFINED FEDERAL,WA,Yakima,53077.0,Yakima County


In [5]:
counties

,geo_point_2d,statefp,countyfp,countyns,geoid,name,namelsad,stusab,lsad,classfp,...,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,state_name,countyfp_nozero,geometry
0,"{'lon': -83.6665354077, 'lat': 41.001938375}",39,063,01074044,39063,Hancock,Hancock County,OH,06,H1,...,22300,NaN,A,1376125660,6020647,+41.0004711,-083.6660335,Ohio,63,"POLYGON ((-83.88076 41.08036, -83.88076 41.081..."
1,"{'lon': -77.4574003866, 'lat': 38.4207190191}",51,179,01480173,51179,Stafford,Stafford County,VA,06,H1,...,47900,47894,A,697237202,28088011,+38.4132608,-077.4513342,Virginia,179,"POLYGON ((-77.58878 38.50404, -77.58295 38.508..."
2,"{'lon': -98.4999473753, 'lat': 40.1764332141}",31,181,00835912,31181,Webster,Webster County,NE,06,H1,...,NaN,NaN,A,1489017962,337241,+40.1806461,-098.4985897,Nebraska,181,"POLYGON ((-98.72662 40.29186, -98.72663 40.292..."
3,"{'lon': -99.756726125, 'lat': 28.4225359982}",48,127,01383849,48127,Dimmit,Dimmit County,TX,06,H1,...,NaN,NaN,A,3441795367,14576535,+28.4235871,-099.7658713,Texas,127,"POLYGON ((-99.78085 28.64383, -99.78079 28.643..."
4,"{'lon': -91.1324230468, 'lat': 41.7723266607}",19,031,00465205,19031,Cedar,Cedar County,IA,06,H1,...,NaN,NaN,A,1500798712,6385832,+41.7723600,-091.1326105,Iowa,31,"POLYGON ((-91.36608 41.86007, -91.36620 41.860..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,"{'lon': 145.777379252, 'lat': 15.1992652149}",69,110,01805247,69110,Saipan,Saipan Municipality,MP,12,H1,...,NaN,NaN,A,118890997,532767361,+15.1980954,+145.7771923,Northern Mariana Islands,110,"POLYGON ((145.58775 15.12775, 145.63317 15.135..."
3229,"{'lon': -100.394202624, 'lat': 40.5300832055}",31,063,00835853,31063,Frontier,Frontier County,NE,06,H1,...,NaN,NaN,A,2524283299,14110075,+40.5309469,-100.4066834,Nebraska,63,"POLYGON ((-100.78193 40.61298, -100.78193 40.6..."
3230,"{'lon': -94.2741206259, 'lat': 35.1993218755}",05,131,00069181,05131,Sebastian,Sebastian County,AR,06,H1,...,22900,NaN,A,1375720610,37325387,+35.1969808,-094.2749889,Arkansas,131,"POLYGON ((-94.43391 35.31996, -94.43391 35.320..."
3231,"{'lon': -97.5182740096, 'lat': 28.0087830727}",48,409,01383990,48409,San Patricio,San Patricio County,TX,06,H1,...,18580,NaN,A,1795990944,37577389,+28.0117944,-097.5171566,Texas,409,"POLYGON ((-97.63139 27.89171, -97.63132 27.891..."


In [6]:
counties.geometry

0       POLYGON ((-83.88076 41.08036, -83.88076 41.081...
1       POLYGON ((-77.58878 38.50404, -77.58295 38.508...
2       POLYGON ((-98.72662 40.29186, -98.72663 40.292...
3       POLYGON ((-99.78085 28.64383, -99.78079 28.643...
4       POLYGON ((-91.36608 41.86007, -91.36620 41.860...
                              ...                        
3228    POLYGON ((145.58775 15.12775, 145.63317 15.135...
3229    POLYGON ((-100.78193 40.61298, -100.78193 40.6...
3230    POLYGON ((-94.43391 35.31996, -94.43391 35.320...
3231    POLYGON ((-97.63139 27.89171, -97.63132 27.891...
3232    POLYGON ((-90.47424 30.03135, -90.47003 30.031...
Name: geometry, Length: 3233, dtype: geometry

In [7]:
fires.shape

(2303566, 40)

In [8]:
fires.columns

Index(['X', 'Y', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE',
       'NWCG_CAUSE_AGE_CATEGORY', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME',
       'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR',
       'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME'],
      dtype='object')

In [9]:
fires = fires.drop(columns=['X', 'Y', 'OBJECTID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE',
       'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME',
       'NWCG_CAUSE_AGE_CATEGORY'])
#rename columns if needed - make them lower cases
fires.columns = map(str.lower, fires.columns)

In [10]:
#filter data to only California
calfires = fires[fires['state'] == 'CA']
calfires

,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,discovery_time,nwcg_cause_classification,nwcg_general_cause,cont_date,...,cont_time,fire_size,fire_size_class,latitude,longitude,owner_descr,state,county,fips_code,fips_name
0,1,FS-1418826,FOUNTAIN,2005,2005/02/02 00:00:00+00,33,1300.0,Human,Power generation/transmission/distribution,2005/02/02 00:00:00+00,...,1730.0,0.10,A,40.036944,-121.005833,USFS,CA,63,6063.0,Plumas County
1,2,FS-1418827,PIGEON,2004,2004/05/12 00:00:00+00,133,845.0,Natural,Natural,2004/05/12 00:00:00+00,...,1530.0,0.25,A,38.933056,-120.404444,USFS,CA,61,6061.0,Placer County
2,3,FS-1418835,SLACK,2004,2004/05/31 00:00:00+00,152,1921.0,Human,Debris and open burning,2004/05/31 00:00:00+00,...,2024.0,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17,6017.0,El Dorado County
3,4,FS-1418845,DEER,2004,2004/06/28 00:00:00+00,180,1600.0,Natural,Natural,2004/07/03 00:00:00+00,...,1400.0,0.10,A,38.559167,-119.913333,USFS,CA,3,6003.0,Alpine County
4,5,FS-1418847,STEVENOT,2004,2004/06/28 00:00:00+00,180,1600.0,Natural,Natural,2004/07/03 00:00:00+00,...,1200.0,0.10,A,38.559167,-119.933056,USFS,CA,3,6003.0,Alpine County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303542,400732952,ICS209_2019_10757785,MARIA,2019,2019/10/31 00:00:00+00,304,2115.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,9999.00,G,34.337222,-119.053333,MISSING/NOT SPECIFIED,CA,Ventura,6111.0,Ventura County
2303543,400732953,ICS209_2019_10762771,BELMONT,2019,2019/05/29 00:00:00+00,149,1900.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,835.00,E,35.307500,-119.964444,MISSING/NOT SPECIFIED,CA,San Luis Obispo,6079.0,San Luis Obispo County
2303551,400732962,ICS209_2019_10781965,ROCK,2019,2019/06/25 00:00:00+00,176,2230.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,2422.00,F,37.472222,-121.249444,MISSING/NOT SPECIFIED,CA,Stanislaus,6099.0,Stanislaus County
2303556,400732973,ICS209_2019_10802166,WALKER,2019,2019/09/04 00:00:00+00,247,1446.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,54608.00,G,40.053250,-120.668900,MISSING/NOT SPECIFIED,CA,Plumas,6063.0,Plumas County


In [11]:
calfires.dtypes

fod_id                         int64
fpa_id                        object
fire_name                     object
fire_year                      int64
discovery_date                object
discovery_doy                  int64
discovery_time               float64
nwcg_cause_classification     object
nwcg_general_cause            object
cont_date                     object
cont_doy                     float64
cont_time                    float64
fire_size                    float64
fire_size_class               object
latitude                     float64
longitude                    float64
owner_descr                   object
state                         object
county                        object
fips_code                    float64
fips_name                     object
dtype: object

In [12]:
#check if FIPS code is null for any rows
calfires[calfires['fips_code'].isnull() == True]

,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,discovery_time,nwcg_cause_classification,nwcg_general_cause,cont_date,...,cont_time,fire_size,fire_size_class,latitude,longitude,owner_descr,state,county,fips_code,fips_name
7,8,FS-1418854,SLATE,2005,2005/03/08 00:00:00+00,67,1300.0,Human,Debris and open burning,2005/03/08 00:00:00+00,...,1600.0,0.8,B,40.968056,-122.433889,STATE OR PRIVATE,CA,NaN,NaN,NaN
8,9,FS-1418856,SHASTA,2005,2005/03/15 00:00:00+00,74,1200.0,Human,Debris and open burning,2005/03/15 00:00:00+00,...,1700.0,1.0,B,41.233611,-122.283333,STATE OR PRIVATE,CA,NaN,NaN,NaN
76,78,FS-1419054,LINTO,2005,2005/06/27 00:00:00+00,178,1007.0,Human,Debris and open burning,2005/06/27 00:00:00+00,...,1130.0,0.1,A,41.037222,-123.533056,USFS,CA,NaN,NaN,NaN
87,89,FS-1419071,CANYON,2005,2005/04/24 00:00:00+00,114,710.0,Human,Other causes,2005/04/24 00:00:00+00,...,745.0,0.1,A,34.174444,-117.765000,USFS,CA,NaN,NaN,NaN
88,90,FS-1419073,FOUNTAIN,2005,2005/07/03 00:00:00+00,184,1650.0,Human,Fireworks,2005/07/03 00:00:00+00,...,1800.0,0.5,B,34.153889,-117.841111,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282743,400627586,SFO-2020CACDFMMU001183,FLOWER ST DELHI,2020,2020/06/09 00:00:00+00,161,2107.0,Human,Arson/incendiarism,NaN,...,NaN,1.1,B,37.436310,-120.790430,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN
2282744,400627587,SFO-2020CACDFMMU001633,HWY 165 STEVENSON,2020,2020/07/30 00:00:00+00,212,214.0,Human,Arson/incendiarism,NaN,...,NaN,5.5,B,37.325745,-120.851151,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN
2282747,400627590,SFO-2020CACDFTCU000540,PINE ST VALLEY_SPRINGS,2020,2020/05/21 00:00:00+00,142,2316.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,0.5,B,38.190299,-120.831459,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN
2282748,400627591,SFO-2020CACDFNEU002696,WOLF CREEK RD NCCFD88,2020,2020/11/10 00:00:00+00,315,2106.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,NaN,0.1,A,39.106624,-121.039022,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN


In [13]:
#check for duplicates - no duplicates
calfires[calfires.duplicated()]

,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,discovery_time,nwcg_cause_classification,nwcg_general_cause,cont_date,...,cont_time,fire_size,fire_size_class,latitude,longitude,owner_descr,state,county,fips_code,fips_name


In [14]:
#check if latitude, longitude have null values
calfires[['latitude', 'longitude']].isnull().values.any()

False

In [15]:
geometry = [Point(xy) for xy in zip(calfires["longitude"], calfires["latitude"])]
crs = {'init': 'epsg:4326'}
calfires_geo = gpd.GeoDataFrame(calfires, crs=crs, geometry=geometry)

# https://stackoverflow.com/questions/59707946/merging-latitude-and-longitude-from-separate-columns-in-a-dataframe-then-use-hav

C:\Users\susmi\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [16]:
calfires_geo
# geometry = [Point(xy) for xy in zip(calfires["longitude"], calfires["latitude"])]
# crs = {'init': 'epsg:4326'}
# calfires_geo = gpd.GeoDataFrame(calfires, crs=crs, geometry=geometry)

# geometry = [Point(xy) for xy in zip(df["pickup_longitude"], df["pickup_latitude"])]
# crs = {'init': 'epsg:4326'}
# geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)
#calfires['coordinates'] = calfires[['latitude', 'longitude']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

#df['Colors'] = df[['Black', 'Red', 'Blue', 'Green']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,discovery_time,nwcg_cause_classification,nwcg_general_cause,cont_date,...,fire_size,fire_size_class,latitude,longitude,owner_descr,state,county,fips_code,fips_name,geometry
0,1,FS-1418826,FOUNTAIN,2005,2005/02/02 00:00:00+00,33,1300.0,Human,Power generation/transmission/distribution,2005/02/02 00:00:00+00,...,0.10,A,40.036944,-121.005833,USFS,CA,63,6063.0,Plumas County,POINT (-121.00583 40.03694)
1,2,FS-1418827,PIGEON,2004,2004/05/12 00:00:00+00,133,845.0,Natural,Natural,2004/05/12 00:00:00+00,...,0.25,A,38.933056,-120.404444,USFS,CA,61,6061.0,Placer County,POINT (-120.40444 38.93306)
2,3,FS-1418835,SLACK,2004,2004/05/31 00:00:00+00,152,1921.0,Human,Debris and open burning,2004/05/31 00:00:00+00,...,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17,6017.0,El Dorado County,POINT (-120.73556 38.98417)
3,4,FS-1418845,DEER,2004,2004/06/28 00:00:00+00,180,1600.0,Natural,Natural,2004/07/03 00:00:00+00,...,0.10,A,38.559167,-119.913333,USFS,CA,3,6003.0,Alpine County,POINT (-119.91333 38.55917)
4,5,FS-1418847,STEVENOT,2004,2004/06/28 00:00:00+00,180,1600.0,Natural,Natural,2004/07/03 00:00:00+00,...,0.10,A,38.559167,-119.933056,USFS,CA,3,6003.0,Alpine County,POINT (-119.93306 38.55917)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303542,400732952,ICS209_2019_10757785,MARIA,2019,2019/10/31 00:00:00+00,304,2115.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,9999.00,G,34.337222,-119.053333,MISSING/NOT SPECIFIED,CA,Ventura,6111.0,Ventura County,POINT (-119.05333 34.33722)
2303543,400732953,ICS209_2019_10762771,BELMONT,2019,2019/05/29 00:00:00+00,149,1900.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,835.00,E,35.307500,-119.964444,MISSING/NOT SPECIFIED,CA,San Luis Obispo,6079.0,San Luis Obispo County,POINT (-119.96444 35.30750)
2303551,400732962,ICS209_2019_10781965,ROCK,2019,2019/06/25 00:00:00+00,176,2230.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,2422.00,F,37.472222,-121.249444,MISSING/NOT SPECIFIED,CA,Stanislaus,6099.0,Stanislaus County,POINT (-121.24944 37.47222)
2303556,400732973,ICS209_2019_10802166,WALKER,2019,2019/09/04 00:00:00+00,247,1446.0,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,...,54608.00,G,40.053250,-120.668900,MISSING/NOT SPECIFIED,CA,Plumas,6063.0,Plumas County,POINT (-120.66890 40.05325)


In [17]:
calcounties = counties[counties['state_name'] == 'California']
calcounties

,geo_point_2d,statefp,countyfp,countyns,geoid,name,namelsad,stusab,lsad,classfp,...,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,state_name,countyfp_nozero,geometry
48,"{'lon': -118.729900424, 'lat': 35.3428473738}",06,029,02054176,06029,Kern,Kern County,CA,06,H1,...,12540,NaN,A,21062540828,78680980,+35.3466288,-118.7295064,California,29,"POLYGON ((-119.91366 35.43926, -119.92327 35.4..."
138,"{'lon': -119.815502709, 'lat': 36.0753604529}",06,031,00277280,06031,Kings,Kings County,CA,06,H1,...,25260,NaN,A,3600883803,3162664,+36.0724780,-119.8155301,California,31,"POLYGON ((-119.95892 36.25547, -119.95893 36.2..."
180,"{'lon': -121.074995529, 'lat': 36.6057058971}",06,069,00277299,06069,San Benito,San Benito County,CA,06,H1,...,41940,NaN,A,3596591082,4719300,+36.6116507,-121.0858108,California,69,"POLYGON ((-121.48300 36.76505, -121.48351 36.7..."
191,"{'lon': -119.649322632, 'lat': 36.7581801263}",06,019,00277274,06019,Fresno,Fresno County,CA,06,H1,...,23420,NaN,A,15432139300,136609898,+36.7610058,-119.6550193,California,19,"POLYGON ((-120.42219 36.84014, -120.42172 36.8..."
244,"{'lon': -120.838597596, 'lat': 40.0046824769}",06,063,00277296,06063,Plumas,Plumas County,CA,06,H1,...,NaN,NaN,A,6612400910,156387636,+39.9922953,-120.8243709,California,63,"POLYGON ((-121.36701 40.07767, -121.36688 40.0..."
288,"{'lon': -122.693975573, 'lat': 37.7597150131}",06,075,00277302,06075,San Francisco,San Francisco County,CA,06,H6,...,41860,41884,C,121477743,479114582,+37.7272391,-123.0322294,California,75,"MULTIPOLYGON (((-122.61228 37.81522, -122.5845..."
301,"{'lon': -119.95475358, 'lat': 38.0276014047}",06,109,00277319,06109,Tuolumne,Tuolumne County,CA,06,H1,...,43760,NaN,A,5752029633,138772442,+38.0214344,-119.9647335,California,109,"POLYGON ((-120.50016 38.00412, -120.50025 38.0..."
311,"{'lon': -120.524649929, 'lat': 38.7787375794}",06,017,00277273,06017,El Dorado,El Dorado County,CA,06,H1,...,40900,NaN,A,4423299441,203342316,+38.7856116,-120.5342245,California,17,"POLYGON ((-121.11866 38.71707, -121.11874 38.7..."
432,"{'lon': -120.725028959, 'lat': 41.5898335739}",06,049,00277289,06049,Modoc,Modoc County,CA,06,H1,...,NaN,NaN,A,10225096402,661284408,+41.5929185,-120.7183704,California,49,"POLYGON ((-121.44890 41.47281, -121.44891 41.4..."
451,"{'lon': -118.80047248, 'lat': 36.220156659}",06,107,00277318,06107,Tulare,Tulare County,CA,06,H1,...,47300,NaN,A,12494889125,37207487,+36.2288339,-118.7810551,California,107,"POLYGON ((-118.80373 35.79035, -118.80496 35.7..."


In [18]:
calcounties['fips'] = calcounties.apply(lambda x: f'{x.statefp:0>2}{x.countyfp:0>3}', axis=1)
calcounties['fips'] = calcounties['fips'].astype(float)

In [19]:
calcounties.columns

Index(['geo_point_2d', 'statefp', 'countyfp', 'countyns', 'geoid', 'name',
       'namelsad', 'stusab', 'lsad', 'classfp', 'mtfcc', 'csafp', 'cbsafp',
       'metdivfp', 'funcstat', 'aland', 'awater', 'intptlat', 'intptlon',
       'state_name', 'countyfp_nozero', 'geometry', 'fips'],
      dtype='object')

In [20]:
calcounties[['geometry', 'fips', 'name']].isna().values.any()
calcounties = calcounties.drop(columns=['geo_point_2d', 'statefp', 'countyfp', 'countyns', 'geoid',
       'namelsad', 'stusab', 'lsad', 'classfp', 'mtfcc', 'csafp', 'cbsafp',
       'metdivfp', 'funcstat', 'aland', 'awater', 'intptlat', 'intptlon',
       'state_name', 'countyfp_nozero'])

In [21]:
calcounties

,name,geometry,fips
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0
138,Kings,"POLYGON ((-119.95892 36.25547, -119.95893 36.2...",6031.0
180,San Benito,"POLYGON ((-121.48300 36.76505, -121.48351 36.7...",6069.0
191,Fresno,"POLYGON ((-120.42219 36.84014, -120.42172 36.8...",6019.0
244,Plumas,"POLYGON ((-121.36701 40.07767, -121.36688 40.0...",6063.0
288,San Francisco,"MULTIPOLYGON (((-122.61228 37.81522, -122.5845...",6075.0
301,Tuolumne,"POLYGON ((-120.50016 38.00412, -120.50025 38.0...",6109.0
311,El Dorado,"POLYGON ((-121.11866 38.71707, -121.11874 38.7...",6017.0
432,Modoc,"POLYGON ((-121.44890 41.47281, -121.44891 41.4...",6049.0
451,Tulare,"POLYGON ((-118.80373 35.79035, -118.80496 35.7...",6107.0


In [22]:
calfires_fips = calcounties.sjoin(calfires_geo, how="inner", predicate='intersects')

C:\Users\susmi\anaconda3\lib\site-packages\geopandas\geodataframe.py:2061: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: +init=epsg:4326 +type=crs

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [23]:
calfires_fips

,name,geometry,fips,index_right,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,...,cont_time,fire_size,fire_size_class,latitude,longitude,owner_descr,state,county,fips_code,fips_name
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0,316408,323007,W-522532,GRAPEVINE,2006,2006/06/28 00:00:00+00,179,...,1000.0,0.1,A,35.735000,-117.933000,PRIVATE,CA,Kern,6029.0,Kern County
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0,1894301,400015592,W-704174,INYOKERN,2017,2017/06/10 00:00:00+00,161,...,2000.0,4.0,B,35.739700,-117.840000,PRIVATE,CA,Kern,6029.0,Kern County
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0,315245,321760,W-119542,DITCH,2003,2003/08/14 00:00:00+00,226,...,1830.0,0.3,B,35.749700,-117.819200,BLM,CA,Kern,6029.0,Kern County
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0,217616,220870,W-871,SAND,1999,1999/07/04 00:00:00+00,185,...,2000.0,0.1,A,35.765500,-117.765900,BLM,CA,Kern,6029.0,Kern County
48,Kern,"POLYGON ((-119.91366 35.43926, -119.92327 35.4...",6029.0,317950,324610,W-590743,POND,2009,2009/07/30 00:00:00+00,211,...,1529.0,0.1,A,35.772092,-117.907391,BLM,CA,Inyo,6027.0,Inyo County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,Butte,"POLYGON ((-121.85650 39.53358, -121.85638 39.5...",6007.0,2171175,400504475,FS-6971470,CUDDLEBACK,2019,2019/09/26 00:00:00+00,269,...,2100.0,0.1,A,40.040556,-121.474722,USFS,CA,007,6007.0,Butte County
3201,Butte,"POLYGON ((-121.85650 39.53358, -121.85638 39.5...",6007.0,170220,171918,FS-400616,LOG,2004,2004/07/05 00:00:00+00,187,...,1305.0,0.1,A,40.043889,-121.499167,USFS,CA,NaN,NaN,NaN
3201,Butte,"POLYGON ((-121.85650 39.53358, -121.85638 39.5...",6007.0,11708,11763,FS-1439653,PHILBROOK,2006,2006/08/29 00:00:00+00,241,...,330.0,24.0,C,40.050000,-121.483333,USFS,CA,7,6007.0,Butte County
3201,Butte,"POLYGON ((-121.85650 39.53358, -121.85638 39.5...",6007.0,35907,36095,FS-1477510,COLEMAN,2009,2009/07/05 00:00:00+00,186,...,1130.0,0.1,A,40.062222,-121.483333,STATE OR PRIVATE,CA,7,6007.0,Butte County


In [24]:
calfires_fips['fips'] = calfires_fips['fips'].astype(int)

In [25]:
calfires_fips[['geometry', 'fips', 'name']].isna().values.any()

False

In [26]:
calfires_fips = calfires_fips.drop(columns=['fips_code', 'fips_name', 'county'])
calfires_fips.rename(columns={'name': 'county', 'fips': 'fips_code'}, inplace=True)

In [27]:
calfires_fips[['geometry', 'fips_code', 'county']].isna().values.any()
calfires_fips['fips_code'] = calfires_fips['fips_code'].astype(str)
calfires_fips['discovery_date'] = calfires_fips['discovery_date'].apply(pd.Timestamp)
calfires_fips['cont_date'] = calfires_fips['cont_date'].apply(pd.Timestamp)

In [28]:
calfires_fips.dtypes

county                                    object
geometry                                geometry
fips_code                                 object
index_right                                int64
fod_id                                     int64
fpa_id                                    object
fire_name                                 object
fire_year                                  int64
discovery_date               datetime64[ns, UTC]
discovery_doy                              int64
discovery_time                           float64
nwcg_cause_classification                 object
nwcg_general_cause                        object
cont_date                    datetime64[ns, UTC]
cont_doy                                 float64
cont_time                                float64
fire_size                                float64
fire_size_class                           object
latitude                                 float64
longitude                                float64
owner_descr         

In [29]:
calfires_fips['cont_date'].isna().values.any()

True

In [30]:
calfires_fips = calfires_fips.sort_values(by=['discovery_date'])
calfires_fips['fire_month'] = calfires_fips['discovery_date'].dt.month

In [31]:
calfires_fips

,county,geometry,fips_code,index_right,fod_id,fpa_id,fire_name,fire_year,discovery_date,discovery_doy,...,cont_date,cont_doy,cont_time,fire_size,fire_size_class,latitude,longitude,owner_descr,state,fire_month
2165,Riverside,"POLYGON ((-117.67244 33.87026, -117.67257 33.8...",6065,1135903,1386394,CDF_1992_55_2225_10015,LOCAL,1992,1992-01-01 00:00:00+00:00,1,...,NaT,NaN,NaN,0.20,A,33.663889,-116.171944,MISSING/NOT SPECIFIED,CA,1
503,Calaveras,"POLYGON ((-120.63093 38.34110, -120.63057 38.3...",6009,49219,49564,FS-269504,CITY,1992,1992-01-01 00:00:00+00:00,1,...,1992-01-01 00:00:00+00:00,1.0,210.0,0.10,A,38.205000,-120.335000,STATE OR PRIVATE,CA,1
1661,San Diego,"POLYGON ((-117.43743 33.17953, -117.44954 33.1...",6073,191931,194413,W-465492,VOLCAN 1-5,1992,1992-01-01 00:00:00+00:00,1,...,1992-01-01 00:00:00+00:00,1.0,1700.0,5.00,B,33.166700,-116.634200,BIA,CA,1
2165,Riverside,"POLYGON ((-117.67244 33.87026, -117.67257 33.8...",6065,1135904,1386395,CDF_1992_55_2225_6,STATE,1992,1992-01-02 00:00:00+00:00,2,...,NaT,NaN,NaN,0.50,B,33.896111,-116.990000,MISSING/NOT SPECIFIED,CA,1
1661,San Diego,"POLYGON ((-117.43743 33.17953, -117.44954 33.1...",6073,1128716,1377243,CDF_1992_55_2223_1,COUCH,1992,1992-01-02 00:00:00+00:00,2,...,NaT,NaN,NaN,0.20,A,33.030000,-116.831944,MISSING/NOT SPECIFIED,CA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,Yolo,"POLYGON ((-122.16494 38.64246, -122.16398 38.6...",6113,2262926,400604948,SFO-2020CACDFLNU200800,NaN,2020,2020-12-31 00:00:00+00:00,366,...,NaT,NaN,NaN,0.01,A,38.691903,-121.773416,MISSING/NOT SPECIFIED,CA,12
2816,Placer,"POLYGON ((-121.06544 39.00653, -121.06537 39.0...",6061,2255024,400595872,SFO-2020CACDFNEU003150,RICHARDSON DR PCFD,2020,2020-12-31 00:00:00+00:00,366,...,NaT,NaN,NaN,15.80,C,38.938853,-121.107672,MISSING/NOT SPECIFIED,CA,12
2202,San Mateo,"POLYGON ((-122.58711 37.58755, -122.58679 37.5...",6081,2246256,400585849,SFO-2020CACDFCZU207243,NaN,2020,2020-12-31 00:00:00+00:00,366,...,NaT,NaN,NaN,0.01,A,37.580557,-122.347121,MISSING/NOT SPECIFIED,CA,12
2165,Riverside,"POLYGON ((-117.67244 33.87026, -117.67257 33.8...",6065,2279311,400623664,SFO-2020CACDFRRU017111,COUNTRY RD MOVA,2020,2020-12-31 00:00:00+00:00,366,...,NaT,NaN,NaN,0.10,A,33.982775,-117.234942,MISSING/NOT SPECIFIED,CA,12


In [32]:
# calfires_fips.to_file("calfires1992-2020.geojson", driver="GeoJSON")

In [33]:
calfires_fips.to_file("calfires1992-2020.csv")

C:\Users\susmi\anaconda3\lib\site-packages\fiona\collection.py:646: UserWarning: CSV driver silently converts datetime to string
  warnings.warn(


FionaCPLError: The wrapped function returned an error code, but no error message was set.

In [ ]:
#df = calfires_fips[(calfires_fips['fire_year'] == 1992) & (calfires_fips['fire_month'] == 1)]
df = calfires_fips[calfires_fips["discovery_date"].between('2020-12-01', '2020-12-31')]

In [ ]:
df

In [ ]:
df.plot(column="fire_size", legend = True)

In [ ]:
# base = df.plot(color='white', edgecolor='black')

# df.plot(ax=base, marker='o', color='red', markersize='fire_size')

In [ ]:
k = calfires_fips[calfires_fips['fire_year']==2020]

In [ ]:
k.explore()

In [ ]:
# with open('sectors.json') as f:
#     js = json.load(f)

# # construct point based on lon/lat returned by geocoder
# point = Point(-122.7924463, 45.4519896)

# # check each polygon to see if it contains the point
# for feature in js['features']:
#     polygon = shape(feature['geometry'])
#     if polygon.contains(point):
#         print 'Found containing polygon:', feature